<a href="https://colab.research.google.com/github/mamba124/just_for_fun/blob/classifier-in-autoenc/Denoising_task.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Из кераса загружаем необходимые слои для нейросети
import tensorflow.keras.layers as layers
from tensorflow.keras import backend as K # подтягиваем базовые керасовские функции
from tensorflow.keras.optimizers import Adam # загружаем выбранный оптимизатор
from tensorflow.keras import utils # загружаем утилиты кераса
from google.colab import files # модуль для загрузки файлов в colab
import matplotlib.pyplot as plt # из библиотеки для визуализации данных возьмём интерфейс для построения графиков простых функций
from tensorflow.keras.preprocessing import image # модуль для отрисовки изображения
import numpy as np # библиотека для работы с массивами данных
import librosa #Параметризация аудио
import os 

import pandas as pd # библиотека для анализа и обработки данных
from PIL import Image # модуль для отрисовки изображения
from sklearn.model_selection import train_test_split # модуль для разбивки выборки на тренировочную/тестовую
from sklearn.preprocessing import StandardScaler # модуль для стандартизации данных

from tensorflow.keras.models import Model, load_model #из кераса подгружаем абстрактный класс базовой модели, метод загрузки предобученной модели
from tensorflow.keras.optimizers import RMSprop #из кераса загружаем выбранный оптимизатор

#Автокодировщик

In [0]:
def baseAutoencoder(): # зададим функцию создания базового автокодировщика
    img_input = Input((28,28,1)) # задаём входные размеры

    x = Conv2D(32, (3, 3), padding='same', activation='relu')(img_input) # входные данные передаем на слой двумерной свёртки
    x = BatchNormalization()(x) # затем пропускаем через слой нормализации данных 
    x = Conv2D(32, (3, 3), padding='same', activation='relu')(x) # далее снова слой двумерной свёртки
    x = BatchNormalization()(x) # и еще слой нормализации данных
    x = MaxPooling2D()(x) # передаём на слой подвыборки, снижающий размерность поступивших на него данных

    x = Conv2D(64, (3, 3), padding='same', activation='relu')(x) # передаем на слой двумерной свёртки
    x = BatchNormalization()(x) # пропускаем через слой нормализации данных 
    x = Conv2D(64, (3, 3), padding='same', activation='relu')(x)  # далее снова слой двумерной свёртки
    x = BatchNormalization()(x) # и еще слой нормализации данных
    x = MaxPooling2D()(x) # передаём на слой подвыборки
    # Изображение ужали до 7*7

    x = Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same', activation='relu')(x) # слой разжимает данные(с 7*7 на 14*14)
    x = BatchNormalization()(x) # слой нормализации данных
    
    x = Conv2D(64, (3, 3), padding='same', activation='relu')(x) # передаем на слой двумерной свёртки
    x = BatchNormalization()(x) # слой нормализации данных
    x = Conv2D(64, (3, 3), padding='same', activation='relu')(x) # ещё слой двумерной свёртки
    x = BatchNormalization()(x) # слой нормализации данных
    # Сжатие MaxPooling2D не применяем

    x = Conv2DTranspose(32, (2, 2), strides=(2, 2), padding='same', activation='relu')(x) # слой разжимает данные(с 14*14 на 28*28)
    x = BatchNormalization()(x) # слой нормализации данных
    x = Conv2D(32, (3, 3), padding='same', activation='relu')(x) # передаем на слой двумерной свёртки
    x = BatchNormalization()(x) # слой нормализации данных
    x = Conv2D(32, (3, 3), padding='same', activation='relu')(x) # ещё слой двумерной свёртки
    x = BatchNormalization()(x) # слой нормализации данных

    # Финальный слой двумерной свертки, выдающий итоговое изображение
    x = Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x)

    model = Model(img_input, x) # указываем модель, с оригинальным изображением на входе в сеть и сжатым-разжатым на выходе из сети
    model.compile(optimizer=Adam(),
                  loss='mean_squared_error') # компилируем модель с оптимайзером Адам и среднеквадратичной ошибкой

    return model # функция вернёт заданную модель

# AUDIO

In [4]:
!pip install soundfile

In [0]:
import librosa # for mel-spectrogram estimation
import soundfile # for opening .flac audio
from matplotlib import pyplot as plt
import numpy as np

In [0]:
# normalized log-mel-spectrogram of clean and noisy audios
clean_mel = 1 + np.log(1.e-12 + librosa.feature.melspectrogram(clean_audio, sr=framerate, n_fft=1024, hop_length=256, fmin=20, fmax=8000, n_mels=80)).T / 10.
noisy_mel = 1 + np.log(1.e-12 + librosa.feature.melspectrogram(noisy_audio, sr=framerate, n_fft=1024, hop_length=256, fmin=20, fmax=8000, n_mels=80)).T / 10.

In [0]:
clean_audio, framerate = soundfile.read('20-205-0001.flac')
noisy_audio, framerate = soundfile.read('20-205-0000_noisy.wav')

In [4]:
def get_chunks(signal, length, step):
  batch =[]
  syms_len = len(signal)
  index = 0
  
  while (index + length <= syms_len):
    batch.append(signal[index:index+length])
    index += step

  return batch

def prepare_chunks(audio, len , step):
  batch_list = []
  for item in audio:
    batch_list.append(get_chunks(item))

  return np.array(batch_list)

11.85
11.85


In [0]:
folders = os.listdir('/content/drive/My Drive/train')
clean_arrays = []
noisy_arrays = []

for folder in folders:
  patch = os.listdir(folder)
  for arr in patch:
    clean_arrays.append(np.load(folders+'clean'+folder+arr))
    noisy_arrays.append(np.load(folders+'noisy'+folder+arr))

clean_arrays_ = []
noisy_arrays_ = []

for i in range(len(clean_arrays)):
  clean_arrays_ = [arr for arr in clean_arrays[i]]
  noisy_arrays_ = [arr for arr in noisy_arrays[i]]

In [20]:
length = 60 
step = 25

x_train = prepare_chunks(clean_arrays_, length, step)
y_train = np.ones(x_train.shape)

x_val = prepare_chunks(noisy_arrays_, length, step)
y_val = np.zeros(x_train.shape)

array([-0.47001022, -0.46614806, -0.46428758, ...,  1.47169741,
        1.47291896,  1.479164  ])

In [0]:
def classificator(vector):
  shape_ = (vector.shape[1],vector.shape[2])
  inputs = layers.Input(shape = shape_)
  hidden = layers.Conv1D(8, 3)(inputs)
  hidden = layers.Activation('tanh')(hidden)

  hidden = layers.Flatten()(hidden)
  hidden = layers.Dense(56)(hidden)
  hidden = layers.Activation('relu')(hidden)

  hidden = layers.Dense(100)(hidden)
  out = layers.Dense(1, activation = 'sigmoid')(hidden)
  
  model = Model(inputs = inputs, outputs = out)

  return model

def autoencoder(model, vector):
  #result = model.predict(vector)
  inputs_decision = layers.Input(shape = (1,))
  inputs_vector = layers.Input(shape = (vector.shape[1],vector.shape[2]))
  ##Block1##
  hidden = layers.Conv1D(16, 3) (inputs_vector)
  hidden = layers.Activation('tanh')(hidden)
  hidden = layers.BatchNormalization()(hidden)
  hidden = layers.Conv1D(16, 3)(hidden)
  hidden = layers.Activation('relu')(hidden)
  hidden = layers.BatchNormalization()(hidden)
  hidden = layers.MaxPooling1D(hidden)

  out_class = layers.Flatten()(hidden)

  out_class = layers.Dense(56, activation = 'relu')(out_class)
  out_class = layers.Dense(1, activation = 'sigmoid')(out_class)

  ##Block2##
  hidden = layers.Conv1D(64, 3) (hidden)
  hidden = layers.Activation('tanh')(hidden)
  hidden = layers.BatchNormalization()(hidden)
  hidden = layers.Conv1D(64, 3)(hidden)
  hidden = layers.Activation('relu')(hidden)
  hidden = layers.BatchNormalization()(hidden)
  hidden = layers.MaxPooling1D(hidden)

  
  ##Block3##
  #hidden = Conv1D(128, 3) (hidden)
  #hidden = Activation('tanh')
  #hidden = BatchNormalization()(hidden)
  #hidden = Conv1D(128, 3)(hidden)
  #hidden = Activation('relu')
  #hidden = BatchNormalization()(hidden)
  #hidden = MaxPooling1D(hidden)  

  ##UPBlock1##
  hidden = layers.UpSampling1D(hidden)
  hidden = layers.Conv1D(64, 3, activation= 'relu')(hidden)
  hidden = layers.BatchNormalization()(hidden)
  
  ##UPblock2
  hidden = layers.UpSampling1D(hidden)
  hidden = layers.Conv1D(16, 3, activation= 'relu')(hidden)
  hidden = layers.BatchNormalization()(hidden)

  hidden = layers.Flatten()(hidden)
  hidden = layers.Concatenate([out_class, hidden])
  hidden = layers.Dense(128, activation='relu')(hidden)
  out = layers.Dense(inputs_vector.size)(hidden)

  modelD = Model(inputs=inputs_vector, outputs = out)

  return modelD

  

In [0]:
def shuffle(a, b):
    p = np.random.permutation(len(a)) 
    return a[p], b[p] 

x_train_, y_train_ = shuffle(x_train, y_train)
x_val_, y_val_ = shuffle(x_val, y_val)

In [30]:
classificator = classificator(clean_arrays)
classificator.compile(optimizer = RMSprop(), loss = 'mse', metrics = 'accuracy')
classificator.fit(x_train_, y_train_, batch_size=100, epochs=20, validation_data=x_val_, y_val_)

1/1 [==============================] - 0s 1ms/step - loss: 0.2353 - accuracy: 1.0000


In [0]:
denoise, result = autoencoder(classificator, clean_arrays_)
denoise.compile(optimizer = RMSprop(), loss = 'mse', metrics = 'accuracy')
denoise.fit([result, clean_arrays], clean_arrays)

In [0]:
denoise.predict([])